In [1]:
import sys
import enum
import json
import pandas as pd
import gc


# needs a development version of python-tetris library, since
# TetrioEngine is not included in most recent release
sys.path.append('C:\\proj\\python-tetris')
import tetris

In [2]:
# %run ../tetris-bag.ipynb
# %run ../tetris-field.ipynb

In [3]:

with open('../tetrio-dl/replays/replay.json') as f:
# with open('../tetrio-dl/replays/test-replay-200-frames-garabage-travel.ttrm') as f:
    replay = json.load(f)
    
game = replay['data'][0]
inputs = game['replays'][0]['events']

# replay['data'][0]
# tetris.impl.presets.TetrioEngine()

## Events

### update() method
1. set __subframe to 0__
2. __pull()__: pull event from replay (= get all events for current frame).
3. __advanceFrame()__: frame = frame + 1
4. __ProcessShift()__: handle DAS and ARR
5. __FallEvent()__: gravity stuff
6. __ProcessInterrupts__: ?
7. __CheckObjectiveCleared__: ?



get events at frame. for every event at this frame call:
- `t.im.KeyDown(event)`
- `t.im.KeyUp(event)`
- `t.geh.HandleIGE(event)`
- `t.geh.HandleTargets(event)`

## Detailed event handling
- KeyDown for moveLeft: 
    1. set `lshift` to `true`
    2. (set `lastshift` to `-1`)
    3. set `ldas` to `hoisted ? (das - dcd) : 0`
    4. set `ldasiter` to `arr`
    5. call `processLShift(true, version >= 15 ? 0 : 1)`
    
- KeyUp for moveLeft:
    1. set `lshift` to `false`
    2. set `ldas` to 0
    3. if handling.cancel do 
        - `rdas` = 0
        - `rdasiter` = `arr`
    
#### softdrop
- KeyDown:
    1. `s.falling.keys = -999`
    2. `s.softdrop = true`
- KeyUp:
    1. `s.softdrop = false`

In [4]:
# class InputKey(enum.IntEnum):
#     EMPTY = 0
#     moveLeft = enum.auto()
#     moveRight = enum.auto()
#     rotateCW = enum.auto()
#     rotateCCW = enum.auto()
#     hold = enum.auto()
#     hardDrop = enum.auto()
#     softDrop = enum.auto()
    
    
class KeyState(enum.Enum):
    down = False
    up = True


class KeyStateFrame():
    def __init__(self, state=KeyState.up, frame=-1):
        self.state = state
        self.frame = frame
    
    
class KeyStates():
    def __init__(self):
        self._states = {
            'moveLeft': KeyStateFrame(),
            'moveRight': KeyStateFrame(),
            'softDrop': KeyStateFrame()
        }
    
    def keydown(self, key, frame):
        print(f'[KEYDOWN] {key} at frame {frame}')
        if key not in self._states:
            return
        
        if self._states[key].state == KeyState.down:
            raise Exception(f'key {key} was already pressed')
        
        self._states[key].state = KeyState.down
        self._states[key].frame = frame
    

    def keyup(self, key, frame):
        if key not in self._states:
            return
        
        if self._states[key].state == KeyState.up:
            raise Exception(f'key {key} was not pressed')
        
        # time the key was pressed
        key_downtime = frame - self._states[key].frame
        
        print(f'[KEYUP] {key} at frame {frame}, downtime {key_downtime}')
        
        self._states[key].state = KeyState.up
        
        return key_downtime

In [5]:
class TetrioGame():
    def __init__(self, events_df, seed, options):
        self.events_df = events_df
        self.seed = seed
        self.options = options
        
        self.current_frame = 0
        self.last_event_id = 3  # start with 4th event, since first three are only initialization
        
        self.subframe = 0
                
        self.sim_game = tetris.BaseGame(tetris.impl.presets.TetrioEngine, seed=seed)
        self.key_states = KeyStates()
        
    def __str__(self):
        return str(self.events_df)
    
    def _step(self):
        
        # find events that happened in this frame
        
        events = []
        print('LAST FRAME: ', self.events_df.iloc[self.last_event_id]['frame'])
        
        while self.events_df.iloc[self.last_event_id]['frame'] <= self.current_frame:
            print('LAST ID: ', self.last_event_id)
            events.append(self.events_df.iloc[self.last_event_id])
            self.last_event_id += 1
        
        
        print(f'[EVENTS IN FRAME {self.current_frame}:')
        for e in events:
            self._handle_event(e)
            print(e)
        
        print('===============================================')
        
        self.current_frame += 1
        
    
    def _handle_event(self, event):
        current_frame = event['frame']
        
        if event['type'] == 'keydown' or event['type'] == 'keyup':
            current_key = event['key']
            current_subframe = event['data']['subframe']
        
            # execute action on keydown
            if event['type'] == 'keydown':
                self._execute_action(current_key)
                self.key_states.keydown(current_key, current_frame + current_subframe)
                
            else:
                self.key_states.keyup(current_key, current_frame + current_subframe)
        
        
        elif event['type'] == 'ige':
            print('garbage')
            pass  # garbage

        
#     def _step(self):
#         current_event = self.events_df.iloc[self.current_event_id]
#         current_frame = current_event['frame']
#         # current_time = current_frame * (50/3)
        
#         if current_event['type'] == 'keydown' or current_event['type'] == 'keyup':
#             current_key = current_event['key']
#             current_subframe = current_event['data']['subframe']
            
#             if current_key == 'moveLeft' or current_key == 'moveRight' or current_key == 'softDrop':
#                 # record keyup/keydown
#                 method = self.key_states.keydown if current_event['type'] == 'keydown' else self.key_states.keyup
#                 method(current_key, current_frame + current_subframe)
#             else:
#                 # only execute it if key is pressed
#                 if current_event['type'] == 'keydown':
#                     self._execute_action(current_key)
            
#         elif current_event['type'] == 'ige':
#             print('garbage')
#             pass  # garbage
        
#         print(current_event)
        
#         self.current_event_id += 1
    
    def _execute_action(self, key):
        match key:
            case 'moveLeft':
                self.sim_game.left()
            case 'moveRight':
                self.sim_game.right()
            case 'rotateCW':
                self.sim_game.rotate(1)
            case 'rotateCCW':
                self.sim_game.rotate(-1)
            case 'hold':
                self.sim_game.swap()
            case 'hardDrop':
                self.sim_game.hard_drop()
            case 'softDrop':
                self.sim_game.soft_drop()
    
        
        
        
class TetrioReplay():
    def __init__(self, file_path):
        self.file_path = file_path
        
        with open(file_path) as f:
            replay = json.load(f)
            
        self.games = []
        
        # for each game in the replay
        for game in replay['data']:
            # it's enough to get one seed, since both player share the seed among one game
            seed = self._get_seed(game)
            
            # for each player in this game
            for player in game['replays']:
                # load dataframe
                event_df = pd.DataFrame(player['events'])
                # make key its own column
                event_df['key'] = event_df['data'].apply(lambda d: d['key'] if 'key' in d else None)
                event_df = event_df.loc[(event_df['type'] != 'keyup') | ((event_df['key'] != 'hardDrop') & (event_df['key'] != 'hold') & (event_df['key'] != 'rotateCCW') & (event_df['key'] != 'rotateCW'))]
                
                # extract player options (arr [auto repeat rate], das [delayed auto shift], sdf [soft drop factor]
                options = self._get_options(event_df)
            
                # filter out all other events beside keydown
                # event_df = event_df.loc[event_df['type'] == 'keydown']
                # event_df['data'] = event_df['data'].apply(lambda d: d['key'])
                
                tg = TetrioGame(event_df, seed, options)
                self.games.append(tg)
                
            break
          
        
        # print(len(games))
        # force garbage collection
        del replay
        # gc.collect()
        
    def _get_seed(self, game_data):
        return game_data['replays'][0]['events'][1]['data']['options']['seed']
    
    def _get_options(self, event_data):
        # ARR (Auto Repeat Rate): the speed at which tetrominoes move when holding down movement keys, measured in frames per movement
        # DAS (Delayed Auto Shift): the time between the initial keypress and the start of its automatic repeat movement, measured in frames
        # DCD (DAS Cut Delay): if not 0, any ongoing DAS movement will pause for a set amount of time after dropping/rotating a piece, measured in frames
        # SDF (Soft Drop Factor): the factor with which soft drops change the gravity speed
        return event_data.iloc[1]['data']['options']['handling']
        
        


In [6]:
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output


r = TetrioReplay('../tetrio-dl/replays/replay.json')

for i in range(20):
    r.games[0]._step()
    print(r.games[0].sim_game)

# def clicked(arg):
#     # print("button has been clicked!")
#     clear_output()
#     r.games[0]._step()
#     print(r.games[0].sim_game)

    
# button_download = widgets.Button(description = 'Test Button')
# button_download.on_click(clicked)
# display(button_download)

LAST FRAME:  1
[EVENTS IN FRAME 0:
        Z Z         
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
      @ @           
        @ @         
LAST FRAME:  1
LAST ID:  3
[EVENTS IN FRAME 1:
[KEYDOWN] hold at frame 1.3
frame                                   1
type                              keydown
data     {'key': 'hold', 'subframe': 0.3}
key                                  hold
Name: 3, dtype: object
      J J J         
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
      

In [7]:
events_df = pd.DataFrame(replay['data'][0]['replays'][0]['events'])
# for d in events_df.loc[events_df['type'] == 'ige'].iterrows():
#     print(d[1]['data'])

# print('===================================================================\n' * 3)
    
# events_df = pd.DataFrame(replay['data'][0]['replays'][1]['events'])
# for d in events_df.loc[events_df['type'] == 'ige'].iterrows():
#     print(d[1]['data'])



In [30]:
events_df = pd.DataFrame(replay['data'][0]['replays'][0]['events'])
events_df['key'] = events_df['data'].apply(lambda d: d['key'] if 'key' in d else None)


# events_df.iloc[3]

# events_df = events_df.loc[(events_df['type'] != 'keyup') | ((events_df['key'] != 'hardDrop') & (events_df['key'] != 'hold') & (events_df['key'] != 'rotateCCW') & (events_df['key'] != 'rotateCW'))]
# events_df = events_df.loc[events_df['type'] == 'full']

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(events_df)

print(str(events_df.loc[events_df['type'] == 'full']))





   frame  type                                               data   key
1      0  full  {'successful': False, 'gameoverreason': None, ...  None
